##### **** These pip installs need to be adapted to use the appropriate release level. Alternatively, The venv running the jupyter lab could be pre-configured with a requirement file that includes the right release. Example for transform developers working from git clone:
```
make venv
source venv/bin/activate && pip install jupyterlab
```

In [ ]:
%%capture
## This is here as a reference only
# Users and application developers must use the right tag for the latest from pypi
#!pip install data-prep-toolkit
#!pip install data-prep-toolkit-transforms
#!pip install data-prep-connector

##### ***** Import required Classes and modules

In [ ]:
import ast
import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_python import parse_args
from fdedup_transform_spark import SparkServiceOrchestrator

##### ***** Setup runtime parameters for this transform
We will only provide a description for the parameters used in this example. For a complete list of parameters, please refer to the README.md for this transform:
|parameter:type | value | description |
|-|-|-|
| input_folder:str | \${PWD}/ray/test-data/input/ | folder that contains the input parquet files for the fuzzy dedup algorithm |
| output_folder:str | \${PWD}/ray/output/ | folder that contains the all the intermediate results and the output parquet files for the fuzzy dedup algorithm |
| contents_column:str | contents | name of the column that stores document text |
| document_id_column:str | int_id_column | name of the column that stores document ID |
| num_permutations:int | 112 | number of permutations to use for minhash calculation |
| num_bands:int | 14 | number of bands to use for band hash calculation |
| num_minhashes_per_band | 8 | number of minhashes to use in each band |
| operation_mode:{filter_duplicates,filter_non_duplicates,annotate} | filter_duplicates | operation mode for data cleanup: filter out duplicates/non-duplicates, or annotate duplicate documents |

In [ ]:
# create parameters
input_folder = os.path.join(os.path.abspath(""), "spark", "test-data", "input")
output_folder = os.path.join(os.path.abspath(""), "spark", "output")
params = {
    # transform configuration parameters
    "input_folder": input_folder,
    "output_folder": output_folder,
    "contents_column": "contents",
    "document_id_column": "int_id_column",
    "num_permutations": 112,
    "num_bands": 14,
    "num_minhashes_per_band": 8,
    "operation_mode": "filter_duplicates",
}

##### ***** Use spark runtime to invoke each transform in the fuzzy dedup pipeline

In [ ]:

sys.argv = ParamsUtils.dict_to_req(d=params)
args = parse_args()
# Initialize the orchestrator
orchestrator = SparkServiceOrchestrator(global_params=args)
# Launch spark fuzzy dedup execution
orchestrator.orchestrate()

##### **** The specified folder will include the transformed parquet files.

In [ ]:
import glob
glob.glob("spark/output/cleaned/*")

***** print the input data

In [ ]:
import polars as pl
input_df = pl.read_parquet(os.path.join(os.path.abspath(""), "spark", "test-data", "input", "df1.parquet"))

with pl.Config(fmt_str_lengths=10000000, tbl_rows=-1):
    print(input_df)

***** print the output result

In [ ]:
import polars as pl
output_df = pl.read_parquet(os.path.join(os.path.abspath(""), "spark", "output", "cleaned", "df1.parquet"))
with pl.Config(fmt_str_lengths=10000000, tbl_rows=-1):
    print(output_df)